In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import os

# --- Настройки ---
CLIENT_SECRETS_FILE = 'client_secret_702276937103_6d49n29ltviknhoa2chskeuconq00itm_apps.json'
FILE_ID_TO_DOWNLOAD = '1bNiJHZGmB9eZlJxaSeFNCKjM0Dp9GgVL'
LOCAL_FILENAME = 'downloaded_archive.zip'
TOKEN_FILE = "mycreds.txt"

# --- Функция для аутентификации и получения объекта Google Drive ---
def authenticate_google_drive():
    """
    Аутентифицирует пользователя через Google OAuth и возвращает объект GoogleDrive.
    Использует ручной ввод кода авторизации для большей надежности.
    """
    gauth = GoogleAuth() # Инициализируем GoogleAuth без аргументов

    # Явно указываем PyDrive, где искать файл с секретами
    gauth.settings['client_config_file'] = CLIENT_SECRETS_FILE

    # Проверяем, существует ли файл с секретными ключами
    if not os.path.exists(CLIENT_SECRETS_FILE):
        print(f"!!! Ошибка: Файл '{CLIENT_SECRETS_FILE}' не найден. !!!")
        print("Пожалуйста, создайте его, следуя инструкциям, и убедитесь, что он содержит ваши Client ID и Client Secret.")
        return None

    # Пытаемся загрузить существующий токен
    gauth.LoadCredentialsFile(TOKEN_FILE)

    # Если нет действительных учетных данных, запускаем поток авторизации
    if gauth.credentials is None:
        print("Требуется авторизация.")
        
        # --- РУЧНОЙ ВВОД КОДА АВТОРИЗАЦИИ ---
        # 1. Получаем URL для авторизации
        auth_url = gauth.GetAuthUrl()
        print(f"\n--- Инструкции по авторизации ---")
        print(f"1. Пожалуйста, откройте следующий URL в вашем браузере:")
        print(f"   {auth_url}")
        
        # 2. Просим пользователя ввести код, который он получит после авторизации
        #    Google перенаправит на localhost:порт/, где в URL будет code=...
        #    Например: http://localhost:8090/?code=4%2F...
        #    Нам нужно скопировать часть после code=
        auth_code = input("\n2. Скопируйте часть URL после 'code=' (начиная с '4%2F...') и вставьте сюда:\n   ")
        
        # 3. Обмениваем код на учетные данные
        try:
            gauth.Auth(auth_code)
            print("Авторизация прошла успешно!")
            # Теперь у gauth есть credentials, и мы можем их сохранить.
            gauth.SaveCredentialsFile(TOKEN_FILE)
            print(f"Учетные данные сохранены в '{TOKEN_FILE}'.")
        except Exception as e:
            print(f"Ошибка при авторизации: {e}")
            print("Убедитесь, что вы корректно скопировали код авторизации.")
            return None
        # --- КОНЕЦ РУЧНОГО ВВОДА ---

    elif gauth.access_token_expired:
        print("Токен доступа истек. Обновляем...")
        try:
            gauth.Refresh()
            gauth.SaveCredentialsFile(TOKEN_FILE)
            print(f"Учетные данные обновлены и сохранены в '{TOKEN_FILE}'.")
        except Exception as e:
            print(f"Ошибка при обновлении токена: {e}")
            print("Попробуйте пройти авторизацию заново.")
            # Можно удалить старый токен, чтобы принудительно запустить авторизацию
            if os.path.exists(TOKEN_FILE):
                os.remove(TOKEN_FILE)
            return None
    else:
        print("Используем существующие учетные данные.")

    # Создаем и возвращаем объект GoogleDrive
    drive = GoogleDrive(gauth)
    return drive

# --- Основная часть программы ---
if __name__ == '__main__':
    if FILE_ID_TO_DOWNLOAD == 'ВАШ_ID_АРХИВА':
        print("!!! ВАЖНО: Пожалуйста, замените 'ВАШ_ID_АРХИВА' на реальный ID файла архива из вашего Google Диска.")
    else:
        print("Подключение к Google Drive...")
        drive = authenticate_google_drive()

        if drive:
            print("Подключение успешно установлено!")

            try:
                print(f"Скачивание файла с ID: {FILE_ID_TO_DOWNLOAD}...")
                file_to_download = drive.CreateFile({'id': FILE_ID_TO_DOWNLOAD})
                file_to_download.GetContentFile(LOCAL_FILENAME)
                print(f"Файл успешно скачан как '{LOCAL_FILENAME}'.")

            except Exception as e:
                print(f"Произошла ошибка при скачивании файла: {e}")
                print("Возможные причины:")
                print("  - Неверный ID файла.")
                print("  - У файла нет прав на скачивание (убедитесь, что ссылка на файл доступна для чтения).")
                print("  - Ошибка аутентификации (попробуйте перезапустить ячейку, может потребоваться повторная авторизация).")
        else:
            print("Не удалось подключиться к Google Drive. Проверьте настройки аутентификации.")

/opt/conda/lib/python3.10/site-packages/oauth2client/_helpers.py:255: UserWarning: Cannot access mycreds.txt: No such file or directory
  warnings.warn(_MISSING_FILE_MESSAGE.format(filename))


Подключение к Google Drive...
Требуется авторизация.

--- Инструкции по авторизации ---
1. Пожалуйста, откройте следующий URL в вашем браузере:
   https://accounts.google.com/o/oauth2/auth?client_id=702276937103-6d49n29ltviknhoa2chskeuconq00itm.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code



2. Скопируйте часть URL после 'code=' (начиная с '4%2F...') и вставьте сюда:
    4/0Ab32j90vHypGPMde6Di9JGoF0ahFgwFt0WziiZNuTKoE9O3uAnDXl8REVW68iLihtW2hhw


Authentication successful.
Авторизация прошла успешно!
Учетные данные сохранены в 'mycreds.txt'.
Подключение успешно установлено!
Скачивание файла с ID: 1bNiJHZGmB9eZlJxaSeFNCKjM0Dp9GgVL...
Файл успешно скачан как 'downloaded_archive.zip'.


In [4]:
import zipfile
from pathlib import Path
from tqdm import tqdm

def extract_zip_archive(zip_path, output_dir):
    """
    Разархивирует ZIP-архив в указанную папку с прогресс-баром
    """
    zip_path = Path(zip_path)
    output_dir = Path(output_dir)

    # Проверяем существование архива
    if not zip_path.exists():
        print(f"❌ Архив {zip_path} не найден!")
        return False

    print(f"📦 Начинаем разархивацию: {zip_path} -> {output_dir}")

    try:
        with zipfile.ZipFile(zip_path, 'r') as zipf:
            # Получаем список файлов
            all_files = zipf.namelist()
            print(f"📊 Всего файлов для извлечения: {len(all_files)}")

            # Извлекаем с прогресс-баром
            for file in tqdm(all_files, desc="Разархивация"):
                zipf.extract(file, output_dir)

        print(f"✅ Архив успешно разархивирован!")
        print(f"📍 Файлы находятся в: {output_dir}")

        return True

    except Exception as e:
        print(f"❌ Ошибка при разархивации: {e}")
        return False


# Пример использования
zip_file = "/downloaded_archive.zip"
output_directory = "/workspace/unpacked"

extract_zip_archive(zip_file, output_directory)

📦 Начинаем разархивацию: /downloaded_archive.zip -> /workspace/unpacked
📊 Всего файлов для извлечения: 5671


Разархивация: 100%|██████████| 5671/5671 [00:17<00:00, 319.13it/s] 

✅ Архив успешно разархивирован!
📍 Файлы находятся в: /workspace/unpacked


True

In [5]:
def check_environment():
    import torch
    """Проверка окружения и совместимости"""
    print("=== ПРОВЕРКА ОКРУЖЕНИЯ ===")
    print(f"PyTorch version: {torch.__version__}")
    print(f"CUDA available: {torch.cuda.is_available()}")
    
    if torch.cuda.is_available():
        print(f"CUDA version: {torch.version.cuda}")
        print(f"GPU device: {torch.cuda.get_device_name()}")
        print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    else:
        print("CUDA недоступна! Обучение будет на CPU")

In [6]:
 check_environment()

=== ПРОВЕРКА ОКРУЖЕНИЯ ===
PyTorch version: 2.2.1
CUDA available: True
CUDA version: 12.1
GPU device: NVIDIA GeForce RTX 3090
GPU memory: 23.6 GB


In [21]:
import os
import yaml

def fix_dataset_paths():
    """
    Исправляет пути к датасету созданием символических ссылок
    """
    print("🔧 ИСПРАВЛЕНИЕ ПУТЕЙ ДАТАСЕТА")
    print("═" * 50)
    
    # Правильный путь к данным
    correct_base = "/workspace/unpacked"
    
    # Путь который ищет YOLO
    wrong_base = "/content/drive/MyDrive/cow_detection_cleaned"
    
    # Создаем родительские директории если нужно
    os.makedirs("/content/drive/MyDrive", exist_ok=True)
    
    # Удаляем старую ссылку если существует
    if os.path.exists(wrong_base):
        if os.path.islink(wrong_base):
            os.unlink(wrong_base)
            print(f"🗑️ Удалена старая ссылка: {wrong_base}")
        else:
            print(f"⚠️ Папка уже существует (не ссылка): {wrong_base}")
    
    # Создаем символическую ссылку
    try:
        os.symlink(correct_base, wrong_base)
        print(f"✅ Создана ссылка: {wrong_base} -> {correct_base}")
    except Exception as e:
        print(f"❌ Ошибка создания ссылки: {e}")
        return False
    
    # Проверяем что ссылка работает
    if os.path.exists(wrong_base):
        print("✅ Ссылка работает корректно")
        return True
    else:
        print("❌ Ссылка не работает")
        return False

def create_absolute_config():
    """
    Создает конфиг с абсолютными путями
    """
    print("\n📄 СОЗДАНИЕ КОНФИГА С АБСОЛЮТНЫМИ ПУТЯМИ")
    print("═" * 50)
    
    config_content = {
        'path': '/workspace/unpacked',  # Абсолютный путь
        'train': '/workspace/unpacked/images/train',  # Абсолютные пути
        'val': '/workspace/unpacked/images/val',
        'test': '/workspace/unpacked/images/test',
        'nc': 1,
        'names': ['cow']
    }
    
    config_path = '/workspace/unpacked/dataset_absolute.yaml'
    
    try:
        with open(config_path, 'w') as f:
            yaml.dump(config_content, f, sort_keys=False, default_flow_style=False)
        
        print(f"✅ Конфиг создан: {config_path}")
        
        # Показываем содержимое
        print("\n📄 СОДЕРЖИМОЕ КОНФИГА:")
        print("═" * 40)
        with open(config_path, 'r') as f:
            print(f.read())
        print("═" * 40)
        
        return config_path
        
    except Exception as e:
        print(f"❌ Ошибка: {e}")
        return None

def verify_data_access():
    """
    Проверяет доступ к данным по разным путям
    """
    print("\n🔍 ПРОВЕРКА ДОСТУПА К ДАННЫМ")
    print("═" * 50)
    
    test_paths = [
        '/workspace/unpacked/images/val',
        '/content/drive/MyDrive/cow_detection_cleaned/images/val'
    ]
    
    for path in test_paths:
        exists = os.path.exists(path)
        status = "✅" if exists else "❌"
        print(f"{status} {path}")
        
        if exists:
            files = [f for f in os.listdir(path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            print(f"   📸 изображений: {len(files)}")
            
            if files:
                # Показываем пример файла
                sample_file = os.path.join(path, files[0])
                print(f"   📄 пример: {files[0]} ({os.path.getsize(sample_file) / 1024:.1f} KB)")

# Запускаем исправление
fix_dataset_paths()
create_absolute_config()
verify_data_access()

🔧 ИСПРАВЛЕНИЕ ПУТЕЙ ДАТАСЕТА
══════════════════════════════════════════════════
✅ Создана ссылка: /content/drive/MyDrive/cow_detection_cleaned -> /workspace/unpacked
✅ Ссылка работает корректно

📄 СОЗДАНИЕ КОНФИГА С АБСОЛЮТНЫМИ ПУТЯМИ
══════════════════════════════════════════════════
✅ Конфиг создан: /workspace/unpacked/dataset_absolute.yaml

📄 СОДЕРЖИМОЕ КОНФИГА:
════════════════════════════════════════
path: /workspace/unpacked
train: /workspace/unpacked/images/train
val: /workspace/unpacked/images/val
test: /workspace/unpacked/images/test
nc: 1
names:
- cow

════════════════════════════════════════

🔍 ПРОВЕРКА ДОСТУПА К ДАННЫМ
══════════════════════════════════════════════════
✅ /workspace/unpacked/images/val
   📸 изображений: 563
   📄 пример: DJI_0044_MOV-108_jpg.rf.8a55d148ed208b2e26ee77e6f38a2aac.jpg (1028.2 KB)
✅ /content/drive/MyDrive/cow_detection_cleaned/images/val
   📸 изображений: 563
   📄 пример: DJI_0044_MOV-108_jpg.rf.8a55d148ed208b2e26ee77e6f38a2aac.jpg (1028.2 KB)


In [1]:
# train.py
import os
import torch
from ultralytics import YOLO

def main():
    DATASET_YAML = r"/workspace/unpacked/dataset_fixed.yaml"
    MODEL_NAME = "yolo11n.pt"

    PROJECT_DIR = r"./runs_yolo_11n_v3"
    EXP_NAME = "exp_yolo_aug_11n_v3"

    epochs = 300
    imgsz = 640
    batch = 16
    workers = os.cpu_count() // 2 if os.cpu_count() else 4

    if torch.cuda.is_available():
        device = 0
        print(f"🚀 Training will run on CUDA ({torch.cuda.get_device_name(0)})")
    else:
        device = "cpu"
        print("⚠ Training will run on CPU (CUDA not available)")

    model = YOLO(MODEL_NAME)

    results = model.train(
        data=DATASET_YAML,
        epochs=epochs,
        imgsz=imgsz,
        batch=batch,
        device=device,
        project=PROJECT_DIR,
        name=EXP_NAME,
        workers=workers,
        pretrained=True,
        optimizer="auto",
        patience=20,
        cos_lr=True,
        verbose=True,
        amp=True,
        save_period=10,

        lr0=0.01,           # начальный learning rate
        lrf=0.1,           # конечный learning rate (lr0 * lrf)
        momentum=0.937,      # momentum
        weight_decay=0.0005, # вес decay
        warmup_epochs=3.0,   # эпох прогрева
        warmup_momentum=0.8, # momentum прогрева
        warmup_bias_lr=0.1,
        box=2.2,                   # 🔽 НОВОЕ: вес для box loss
        cls=0.15,                   # 🔽 НОВОЕ: вес для class loss  
        dfl=1.7 )          # яркость

         

    metrics = model.val(
        data=DATASET_YAML,
        imgsz=imgsz,
        device=device,
        split="val",
        verbose=True
    )
    print("Val metrics:", metrics)

    onnx_path = model.export(format="onnx", imgsz=imgsz)
    torchscript_path = model.export(format="torchscript", imgsz=imgsz)
    print("Exported:", onnx_path, torchscript_path)

if __name__ == "__main__":
    main()

🚀 Training will run on CUDA (NVIDIA GeForce RTX 3090)
New https://pypi.org/project/ultralytics/8.3.228 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.227 🚀 Python-3.10.13 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24135MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=2.2, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.15, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/workspace/unpacked/dataset_fixed.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.7, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.1, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.9

In [ ]:
# 🔽 Аугментации
        degrees=10,        # случайные повороты ±10°
        translate=0.1,     # сдвиги до 10%
        scale=0.5,         # масштабирование (0.5–1.5)
        shear=2.0,         # сдвиг по оси
        flipud=0.0,        # вертикальный флип (0 = выкл)
        fliplr=0.5,        # горизонтальный флип (50% вероятности)
        mosaic=1.0,        # mosaic-ауга (по умолчанию включена)
        mixup=0.2,         # mixup (20% вероятности)
        hsv_h=0.015,       # изменение оттенка
        hsv_s=0.7,         # насыщенность
        hsv_v=0.4,
,  
        
        
        
        box=5.0,                   # 🔽 НОВОЕ: вес для box loss
        cls=0.3,                   # 🔽 НОВОЕ: вес для class loss  
        dfl=1